In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# make deterministic
from mingpt.utils import set_seed
set_seed(42)

In [4]:
import os
import time
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt
import argparse
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from data.othello import Othello, OthelloBoardState
from mingpt.dataset import CharDataset
from mingpt.model import GPT, GPTConfig, GPTforProbing
from mingpt.probe_trainer import Trainer, TrainerConfig
from mingpt.probe_model import BatteryProbeClassification, BatteryProbeClassificationTwoLayer

In [73]:
model_ckpt = "playertype/playertype_e40"
probe_root = "playertype"
probe_type = "player" # player or state
probe_layer = 8
twolayer = False
epo = 16
mid_dim = 256

In [74]:
folder_name = f"{probe_root}/probes/{probe_type}"

if twolayer:
    folder_name = folder_name + f"_tl{mid_dim}"  # tl for probes without batchnorm

print(f"Running experiment for {folder_name}")
othello = Othello(data_root="othello_1player", n_games=10000, deduplicate=False, test_split=0)

player_types, games = zip(*othello)
train_dataset = CharDataset(games)


  0%|          | 0/50 [00:00<?, ?it/s]

Mem Used: 8.717 GB:   0%|          | 0/50 [00:00<?, ?it/s]

Running experiment for playertype/probes/player
Loaded 100000 from 1 files
Using 10000 for training, 0 for validation
Dataset created has 10000 sequences, 61 unique words.


In [75]:
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size, n_layer=8, n_head=8, n_embd=512)
model = GPTforProbing(mconf, probe_layer=probe_layer)
# if args.random:
#     model.apply(model._init_weights)
if model_ckpt:  # trained on synthetic dataset
    load_res = model.load_state_dict(torch.load(f"./ckpts/{model_ckpt}.ckpt"))
else:
    raise Exception("not given ckpt path or random flag")

if torch.cuda.is_available():
    device = torch.cuda.current_device()
    model = model.to(device)

In [76]:
# creating dataset of activations and properties
loader = DataLoader(train_dataset, shuffle=False, pin_memory=True, batch_size=1, num_workers=1)
act_container = []
property_container = []

for i, (x, y) in tqdm(enumerate(loader), total=len(loader)):
    tbf = [train_dataset.itos[_] for _ in x.tolist()[0]]
    # truncates game if it is less than 60 moves
    valid_until = tbf.index(-100) if -100 in tbf else 999
    # gets activations for each move
    a = OthelloBoardState()
    act = model(x.to(device))[0, ...].detach().cpu()  # [block_size, f]
    act_container.extend([_[0] for _ in act.split(1, dim=0)[:valid_until]])

    if probe_type == "state":
        properties = a.get_gt(tbf[:valid_until], "get_state")  # [block_size, ]
    elif probe_type == "player":
        properties = [[player_types[i]] for _ in range(len(tbf[:valid_until]))]
    property_container.extend(properties)

    assert len(act_container) == len(property_container)


100%|██████████| 10000/10000 [01:15<00:00, 132.26it/s]


In [77]:
# 3 for board state (empty/black/white), 4 for player type (corners)
probe_class = 4 
# 64 for board state, 1 for player type
num_task = 1 

if twolayer:
    probe = BatteryProbeClassificationTwoLayer(device, probe_class=probe_class, num_task=num_task, mid_dim=mid_dim)
else:
    probe = BatteryProbeClassification(device, probe_class=probe_class, num_task=num_task)

In [78]:
class ProbingDataset(Dataset):
    def __init__(self, act, y):
        assert len(act) == len(y)
        print(f"{len(act)} pairs loaded...")
        self.act = act
        self.y = y
        # print(np.sum(np.array(y)==0), np.sum(np.array(y)==1), np.sum(np.array(y)==2))
        print("y:", np.unique(y, return_counts=True))
        
    def __len__(self, ):
        return len(self.y)
    def __getitem__(self, idx):
        return self.act[idx], torch.tensor(self.y[idx]).to(torch.long)

probing_dataset = ProbingDataset(act_container, property_container)
train_size = int(0.8 * len(probing_dataset))
test_size = len(probing_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(probing_dataset, [train_size, test_size])

585823 pairs loaded...
y: (array([0, 1, 2, 3]), array([150654, 143013, 146761, 145395]))


In [79]:
max_epochs = epo
t_start = time.strftime("_%Y%m%d_%H%M%S")
tconf = TrainerConfig(
    max_epochs=max_epochs, 
    batch_size=1024, 
    learning_rate=1e-3,
    betas=(.9, .999), 
    lr_decay=True, 
    warmup_tokens=len(train_dataset)*5, 
    final_tokens=len(train_dataset)*max_epochs,
    num_workers=0, 
    weight_decay=0., 
    ckpt_path=os.path.join("./ckpts/", folder_name, f"layer{probe_layer}")
)
trainer = Trainer(probe, train_dataset, test_dataset, tconf)

In [80]:
trainer.train(prt=True)
trainer.save_traces()
trainer.save_checkpoint()

epoch 1: train loss 1.57106; lr 1.00e-03; train acc 53.32%:   1%|▏         | 6/458 [00:00<00:08, 54.33it/s]

Decayed: {'proj.weight', 'proj.bias'}


epoch 1: train loss 1.49397; lr 1.00e-03; train acc 59.38%:   1%|▏         | 6/458 [00:00<00:08, 54.33it/s]

epoch 1: train loss 0.11802; lr 1.00e-03; train acc 98.12%: 100%|██████████| 458/458 [00:06<00:00, 75.38it/s]
epoch 2: train loss 0.07059; lr 1.00e-03; train acc 97.83%: 100%|██████████| 458/458 [00:05<00:00, 85.56it/s]
epoch 3: train loss 0.06960; lr 1.00e-03; train acc 98.12%: 100%|██████████| 458/458 [00:05<00:00, 85.19it/s]
epoch 4: train loss 0.06870; lr 1.00e-03; train acc 98.26%: 100%|██████████| 458/458 [00:06<00:00, 75.90it/s]
epoch 5: train loss 0.06871; lr 1.00e-03; train acc 97.97%: 100%|██████████| 458/458 [00:05<00:00, 89.13it/s]
epoch 6: train loss 0.06641; lr 7.50e-04; train acc 97.97%: 100%|██████████| 458/458 [00:05<00:00, 87.27it/s]
epoch 7: train loss 0.06606; lr 7.50e-04; train acc 98.41%: 100%|██████████| 458/458 [00:05<00:00, 77.89it/s]
epoch 8: train loss 0.06506; lr 5.63e-04; train acc 97.83%: 100%|██████████| 458/458 [00:05<00:00, 86.92it/s]
epoch 9: train loss 0.06352; lr 4.22e-04; train acc 98.26%: 100%|██████████| 458/458 [00:05<00:00, 87.98it/s]
epoch 10: 

### trying to debug probes

In [81]:
probe = BatteryProbeClassification(device, probe_class=4, num_task=1)
load_res = probe.load_state_dict(torch.load(f"./ckpts/playertype/probes/player/layer6/checkpoint.ckpt"))

probe = probe.to(device)

In [82]:
train_loader = DataLoader(train_dataset, shuffle=False, pin_memory=True, batch_size=128, num_workers=1)

total_states = 0
total_correct = 0

pbar = tqdm(enumerate(train_loader), total=len(train_loader))
for it, (x, y) in pbar:
    x = x.to(device)  # [B, f]
    y = y.to(device)  # [B, #task=1] 

    logits, loss = probe(x, y)
    pred = torch.argmax(logits, dim=2)
    pred = pred.detach().cpu().numpy()

    true = y.detach().cpu().numpy()
    correct = pred == true
    total_correct += np.count_nonzero(correct)

    total_states += x.shape[0]

    

100%|██████████| 3662/3662 [00:07<00:00, 488.84it/s]


In [83]:
print(total_correct, total_states)
print(total_correct / total_states)

446590 468658
0.9529123582655156
